In [1]:
#  https://github.com/medspacy/medspacy
# python -m spacy download en_core_web_sm

In [2]:
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2

# import medspacy
# from medspacy.ner import TargetRule
# from medspacy.visualization import visualize_ent

In [3]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [4]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [5]:
el_id = "04d7d459-e712-40bd-80e4-6b5255f2f44a"
sql = """
SELECT id, element_text 
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER');
;"""

cur.execute(sql)
print(f"SQL: {sql}")
print("============")
rows = cur.fetchall()
if not len(rows):
    print("empty")
for row in rows:
    # print(row)
    el_id = row[0]
    text = row[1]
    text1 = NER(text)

    for word in text1.ents:
        if word.label_ in ["ORG", "PERSON", "GPE"] and word.text not in ["OpenAI", "AI"]:
            print((word.text, word.label_))
            sql = f"""
            INSERT INTO public.tbl_ner(
                element_id, word_text, word_label)
                VALUES ('{el_id}', '{word.text}', '{word.label_}');
            ;"""
            print(sql)
            cur.execute(sql)
            conn.commit()
            print("============")

SQL: 
SELECT id, element_text 
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER');
;
('VOL', 'ORG')

            INSERT INTO public.tbl_ner(
                element_id, word_text, word_label)
                VALUES ('cfa69fe8-eac0-4842-b338-f0d9db7f7969', 'VOL', 'ORG');
            ;
('Capgemini', 'GPE')

            INSERT INTO public.tbl_ner(
                element_id, word_text, word_label)
                VALUES ('079c8c24-ce4a-422d-905e-a10181b8c23c', 'Capgemini', 'GPE');
            ;
('Foreword', 'ORG')

            INSERT INTO public.tbl_ner(
                element_id, word_text, word_label)
                VALUES ('85437da4-f0c7-4db8-ab16-7e3f661b0a67', 'Foreword', 'ORG');
            ;
('the World Economic Forum', 'ORG')

            INSERT INTO public.tbl_ner(
                element_id, word_text, word_label)
                VALUES ('8f9bf494-2631-4805-b95c-03a20e4a0101', 'the World Economic Forum', 'ORG');
            ;


In [6]:
cur.close()
conn.close()